In [6]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
# import dataset
dataset = pd.read_csv('csv/cleaned_train_all.csv')
dataset.head()
dataset.shape

x = dataset.drop(['credit_card_default'], axis = 1)
y = dataset['credit_card_default'].values
print(x.shape, y.shape)

(43508, 16) (43508,)


In [4]:
x.head()

,customer_id,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months
0,CST_115179,46,0,0.0,1,0.0,107934.04,612.0,1.0,1.0,33070.28,18690.93,73,544,2,1
1,CST_121920,29,1,0.0,1,0.0,109862.62,2771.0,2.0,0.0,15329.53,37745.19,52,857,0,0
2,CST_109330,37,1,0.0,1,0.0,230153.17,204.0,2.0,0.0,48416.60,41598.36,43,650,0,0
3,CST_128288,39,0,0.0,1,0.0,122325.82,11941.0,2.0,0.0,22574.36,32627.76,20,754,0,0
4,CST_151355,46,1,1.0,1,0.0,387286.00,1459.0,1.0,0.0,38282.95,52950.64,75,927,0,0


In [7]:
scalar =  preprocessing.StandardScaler()

# copy of datasets
X_train = x.copy()

# numerical features

#for all 
num_cols = ['net_yearly_income','no_of_days_employed','yearly_debt_payments','credit_limit']

#for less
#num_cols = ["credit_limit_used(%)", "credit_score"]

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = scalar.fit(X_train[[i]])
    
    # transform the training data column
    X_train[i] = scale.transform(X_train[[i]])


In [ ]:
X_train = X_train.set_index('customer_id')


In [10]:
# Splitting Dataset into Training and Test Set
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2  , random_state=2)

In [11]:
model = keras.Sequential()
model.add(keras.Input(shape=(None, 28)))
model.add(layers.SimpleRNN(512, return_sequences=True, activation="relu"))
model.add(layers.SimpleRNN(512, activation="relu"))
model.add(layers.Dense(2))

In [12]:
print(model.summary())
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"],
)

model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(X_test, y_test, batch_size=64, verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, None, 512)         276992    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 512)               524800    
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 802,818
Trainable params: 802,818
Non-trainable params: 0
_________________________________________________________________
None


C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10


ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 213, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer